In [1]:
import tensorflow

In [5]:
import torch
import tqdm
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    
    def forward (self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
    

In [14]:
model = NeuralNetwork().to(device)

In [15]:
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [32]:
X = torch.rand(1,28,28, device=device)

In [33]:
logits = model(X)

In [34]:
pred_prob = nn.Softmax(dim=1)(logits)

In [35]:
y_pred = pred_prob.argmax(1)

In [36]:
y_pred

tensor([2])

In [37]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [43]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
flat_image.shape

torch.Size([3, 784])

In [46]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [50]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 4.0074e-01, -4.9060e-02,  1.8367e-01,  5.3195e-01,  2.8700e-01,
         -5.3820e-01, -3.4845e-01,  3.7501e-01,  2.5566e-01, -1.3479e-01,
          3.1509e-01, -3.0438e-01,  4.5505e-01, -1.9505e-01,  2.9651e-01,
         -1.9751e-02, -1.0258e-02, -3.0992e-01, -9.1907e-02, -8.3434e-03],
        [-1.3046e-01, -4.3443e-01, -7.7687e-05,  1.4657e-01,  5.7243e-01,
         -5.0668e-01, -1.7424e-01,  1.9370e-01,  2.5798e-01, -9.8439e-03,
         -1.4310e-01,  1.7565e-01,  6.5850e-01, -4.7980e-01,  4.6074e-01,
         -1.2130e-01,  6.9148e-02, -1.9052e-01, -3.5253e-01, -2.4710e-01],
        [ 4.2937e-01, -3.8026e-01,  1.9067e-01,  8.5662e-01,  2.7768e-01,
         -5.3431e-01, -2.9389e-01,  4.0502e-01,  7.2211e-03,  7.4831e-03,
          9.7711e-02,  3.4824e-02,  5.7297e-01, -2.0743e-01,  3.8882e-01,
          1.0785e-02,  1.5716e-01, -4.0131e-01,  3.6537e-02, -5.6328e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.4007, 0.0000, 0.1837, 0.5319, 0.2870,

In [52]:
seq_modules = nn.Sequential( flatten, layer1, nn.ReLU(), nn.Linear(20,10))

In [53]:
input_image = torch.rand(3, 28,28)

In [54]:
logits = seq_modules(input_image)

In [56]:
logits.shape

torch.Size([3, 10])

In [57]:
softmax = nn.Softmax(dim=1)

In [58]:
pred = softmax (logits)

In [60]:
pred.argmax()

tensor(15)

In [61]:
print(f"Model structure: {model}\n\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)




In [62]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0139,  0.0281,  0.0030,  ..., -0.0033,  0.0044, -0.0280],
        [-0.0276, -0.0011,  0.0298,  ..., -0.0121,  0.0346, -0.0011]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0007, -0.0166], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0054,  0.0222, -0.0058,  ...,  0.0101, -0.0101, -0.0294],
        [ 0.0375, -0.0213,  0.0273,  ...,  0.0296, -0.0147, -0.0151]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([-0.0009, -0.0233], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[ 0.0137, -0.0125, -0.0018,  ..., -0.0351, -0.0150, -0.0007],
        [-0.0214, -0.0083, -0.0240,  ...,  0.0109,  0.0174, -0.0084]],
       grad_fn=<SliceBackward0>)

In [65]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True) # requires_grad 
b = torch.randn(3, requires_grad=True) # requires_grad 
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

A function that we apply to tensors to construct computational graph is in fact an object of class **Function**. This object knows how to compute the function in the forward direction, and also how to compute its derivative during the backward propagation step. A reference to the backward propagation function is stored in grad_fn property of a tensor. You can find more information of Function in the documentation.

In [66]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7efd81d8ded0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7efd81d8df50>
